In [ ]:
 !pip install minsearch

In [23]:
import os
from google import genai
from dotenv import load_dotenv

load_dotenv()

True

In [1]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [2]:
documents[2]

{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
 'section': 'General course-related questions',
 'question': 'Course - Can I still join the course after the start date?',
 'course': 'data-engineering-zoomcamp'}

In [3]:
from minsearch import AppendableIndex

index = AppendableIndex(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

In [6]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5,
        output_ids=True
    )

    return results

In [13]:
question = "Can I still take the course?"

In [14]:
search_result = search(question)

In [10]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

<QUESTION>
{question}
</QUESTION>

<CONTEXT>
{context}
</CONTEXT>
""".strip()

def build_prompt(query, search_results):
    context = ""

    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [25]:
prompt = build_prompt(question, search_result)

In [28]:
client = genai.Client()

def llm(prompt):
    response = client.models.generate_content(
    model = 'gemini-2.0-flash',
    contents = [prompt]
    )
    return response.text

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [30]:
llm(prompt)

"Yes, you can still take the course and submit homework even if you don't register.\n"

In [39]:
print(rag("How do I run Kafka?"))

To run a Java Kafka producer/consumer/kstreams in the terminal, navigate to the project directory and run: `java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java`.



In [40]:
print(llm("How do I patch KDE under FreeBSD?"))

Patching KDE under FreeBSD is similar to patching other software, but with some KDE-specific considerations. Here's a comprehensive guide:

**1. Understanding the Situation**

*   **Why are you patching?**  Are you applying a security patch, a bug fix, or a custom modification?  Knowing the reason helps you choose the right approach.
*   **Is it a KDE Frameworks, Plasma, or Applications patch?** KDE is organized into these three main areas, each potentially with its own build system and locations.
*   **Where did the patch come from?** Official KDE bug reports, mailing lists, or a third-party source? Verify the source's credibility, especially if it's a security patch.
*   **Do you use ports or packages?** This significantly affects where files are located.

**2. Identifying the Affected Files and Location**

*   **Examine the patch file:** The patch file itself contains critical information. It tells you:
    *   **The file paths:** Look for lines starting with `--- a/` and `+++ b/`. 

# Agentic RAG

In [42]:
prompt_template = """
You're a course teaching assistant.

You're given a QUESTION from a course student and that you need to answer with your own knowledge and provided CONTEXT.
At the beginning the context is EMPTY.

<QUESTION>
{question}
</QUESTION>

<CONTEXT> 
{context}
</CONTEXT>

If CONTEXT is EMPTY, you can use our FAQ database.
In this case, use the following output template:

{{
"action": "SEARCH",
"reasoning": "<add your reasoning here>"
}}

If you can answer the QUESTION using CONTEXT, use this template:

{{
"action": "ANSWER",
"answer": "<your answer>",
"source": "CONTEXT"
}}

If the context doesn't contain the answer, use your own knowledge to answer the question

{{
"action": "ANSWER",
"answer": "<your answer>",
"source": "OWN_KNOWLEDGE"
}}
""".strip()